In [24]:
# TODO: use manual HTML features
import numpy as np
import pandas as pd
from utils import run_ML
from sklearn.metrics import f1_score
from itertools import groupby
import pickle
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
import os

In [25]:
import codecs
import gensim
import urllib
import logging
from pathlib import Path
from bs4 import BeautifulSoup
from numpy import load, save,  asarray
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import networkx as nx
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore")

In [26]:
tmp_dir = "E:/Phising URL Detection/Baseline/DLM-main/DLM-main/tmp"
data_dir = "E:/Phising URL Detection/baseline/HybridDLM-main/HybridDLM-main/dataset_URLs/URLdataset_26.12_html_files"

In [27]:
doc2vec = gensim.models.doc2vec.Doc2Vec.load("E:/Phising URL Detection/Baseline/DLM-main/DLM-main/doc2vec.model")

In [28]:
from utils import generate_filename, convert, read_corpus

In [29]:
def callTag(tag, node):
    node_from = node
    node = node + 1

    for tag in tag.children:
        attr_node = 1
        if (tag.name is not None):
            nodes_list.append(convert([str(node_from), str(node)]))
            attr_list.append(str(node) + ",nname," + str(tag.name))
            attr_list.append(str(node) + ",value," + "")
            for attr in tag.attrs:
                nodes_list.append(convert([str(node), str(node) + "_" + str(attr_node)]))
                attr_list.append(str(node) + "_" + str(attr_node) + ",nname," + str(attr))
                attr_list.append(str(node) + "_" + str(attr_node) + ",value," + str(tag.get(attr)))
                attr_node = attr_node + 1
            callTag(tag, node)

In [30]:
def get_udst_inputs(url, rec_id, phishing_flag):
    node = 0
    f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(f)
    callTag(soup, node)

    nodes_list.pop(0)
    attr_list.pop(1)
    attr_list.insert(1, '1, value, ' + url)

    G = nx.Graph()
    G.add_edges_from(nodes_list)

    A = nx.adjacency_matrix(G) # N*N Adjacency matrix

    df_features = pd.DataFrame(0.0, index=G.nodes(), columns=['nname', 'value'])

    attr_val_list = []
    for x in attr_list:
        y = x.split(',')
        if y[1] == "value":
            attr_val_list.append(y[2])

    for x in attr_list:
        y = x.split(',')
        test_corpus = list(read_corpus([y[2]], tokens_only=True))
        vector = doc2vec.infer_vector(test_corpus[0])
        # print(test_corpus[0])

        if y[1] == "nname":
            df_features.loc[y[0]]["nname"] = vector

        else:
            df_features.loc[y[0]]["value"] = vector

    X = df_features.values # N*d Feature Matrix
    # y = to_categorical(phishing_flag, num_classes=2).tolist() # Label
    y = phishing_flag

    X_ = np.array([np.array(ai, dtype=np.float32) for ai in X.tolist()])

    # url_token = url_tokenizer([url])[0]
    url_token = None

    return X_, A, y, url_token, url

### Load data

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
df=pd.read_csv('URLdataset_13.2_2_final_suf.csv')
df['label'] = df['label'].replace({'legitimate': 0, 'phishing': 1})
smalldata=df
labels=df['label']

In [33]:
smalldata.head(2)

,rec_id,url,label
0,1274,https://www.studiosamo.it/,0
1,2352,http://www.redditery.com/,0


In [34]:
import collections
counter = collections.Counter(labels)
counter, len(labels)

(Counter({0: 1496, 1: 735}), 2231)

### Conventional Models

In [35]:
from utils import extract_features_url

In [36]:
extract_features_url("https://www.klikaj.hr/")

{'domain': 'www.klikaj.hr',
 'num_subdomains': 2,
 'contains_ip': 0,
 'path_length': 1,
 'num_path_segments': 1,
 'uses_https': 1,
 'file_extension': '',
 'count_special_characters': 6,
 'count_non_alphanumeric_characters': 6,
 'TLD': 'hr',
 'count_obfuscated_characters': 0,
 'letter_ratio_in_url': 0.7272727272727273,
 'digit_ratio_in_url': 0.0,
 'count_equals_in_url': 0,
 'NoOfAmpersandInURL': 0,
 'CharContinuationRate': 0.18181818181818182,
 'ratio_obfuscated_characters': 0.0,
 'NoOfQMarkInURL': 0}

In [37]:
## test on numerical URLs features
from utils import extract_numerical_features
phish_url = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
run_ML(np.array(phish_url), labels, "URLdataset13.2", "url_manual_numerical")

Run: 0, fold: 0
LightGBM, Fold 0 - F1 Score: 0.8060715, Precision: 0.8113705, Recall: 0.8015354
Run: 0, fold: 1
LightGBM, Fold 1 - F1 Score: 0.8081386, Precision: 0.8146110, Recall: 0.8028311
Run: 0, fold: 2
LightGBM, Fold 2 - F1 Score: 0.7816684, Precision: 0.7920805, Recall: 0.7744137
Run: 0, fold: 3
LightGBM, Fold 3 - F1 Score: 0.8028584, Precision: 0.7999533, Recall: 0.8060617
Run: 0, fold: 4
LightGBM, Fold 4 - F1 Score: 0.7932360, Precision: 0.8013633, Recall: 0.7874344
['LightGBM']
F1 Score, Precision, Recall: [[0.7983946 0.8038757 0.7944553]]


In [38]:
np.random.seed(0)
n_samples = len(smalldata.index)
train_idx = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
test_idx = list(set(list(range(n_samples))).difference(set(train_idx)))
data_df = np.array(phish_url)
import lightgbm as lgb
model = lgb.LGBMClassifier(verbose=-1)
model.fit(data_df[train_idx], labels[train_idx])
y_predict=model.predict(data_df[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))

0.7878239600609808


### Extract graph features from URLs for PyG

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
# List of URLs
urls = list(smalldata['url'])
# Tokenization and N-grams Generation
# You can adjust ngram_range to extract different n-grams (e.g., (1, 1) for unigrams, (2, 2) for bigrams, etc.)
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 4)) #5
X_counts = vectorizer.fit_transform(urls)
# TF-IDF Transformation
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X_counts)
# Extracted Features
feature_names = vectorizer.get_feature_names_out()
X_counts_data = X_counts.toarray() # not necessary
# Train lgb
model_lgb = lgb.LGBMClassifier(verbose=-1)
model_lgb.fit(X_counts_data[train_idx], labels[train_idx])
y_predict=model_lgb.predict(X_counts_data[test_idx]) 
print(f1_score(y_predict, labels[test_idx], average='macro'))
feature_imp_gain = pd.DataFrame(sorted(zip(model_lgb.booster_.feature_importance(importance_type='gain'),
                                           feature_names), reverse=True), columns=['Value', 'Feature'])
feature_imp_split = pd.DataFrame(sorted(zip(model_lgb.booster_.feature_importance(importance_type='split'),
                                            feature_names), reverse=True), columns=['Value', 'Feature'])
top_ngrams_features = list(set(list(feature_imp_gain.iloc[:200,1]) + list(feature_imp_split.iloc[:200,1])))
cv = CountVectorizer(analyzer='char', ngram_range=(1, 4))
cv.fit(top_ngrams_features)

0.8815119367883189


CountVectorizer(analyzer='char', ngram_range=(1, 4))

In [40]:
def extract_feature_CountVectorizer(model, url):
    return model.transform([url]).toarray().flatten()

In [41]:
# extract countvectorizer feature of all URLs
phish_url_countvectorizer = []
for link in list(smalldata.iloc[:,1]):
    url_features = extract_feature_CountVectorizer(cv,link)
    phish_url_countvectorizer.append(url_features)
run_ML(np.array(phish_url_countvectorizer), labels, "URLdataset13.2", "ngramURLfeatures")

Run: 0, fold: 0
LightGBM, Fold 0 - F1 Score: 0.8896670, Precision: 0.8875771, Recall: 0.8918725
Run: 0, fold: 1
LightGBM, Fold 1 - F1 Score: 0.8962911, Precision: 0.9045304, Recall: 0.8893836
Run: 0, fold: 2
LightGBM, Fold 2 - F1 Score: 0.8898642, Precision: 0.8911486, Recall: 0.8886278
Run: 0, fold: 3
LightGBM, Fold 3 - F1 Score: 0.9097058, Precision: 0.9097058, Recall: 0.9097058
Run: 0, fold: 4
LightGBM, Fold 4 - F1 Score: 0.8694144, Precision: 0.8840336, Recall: 0.8595280
['LightGBM']
F1 Score, Precision, Recall: [[0.8909885 0.8953991 0.8878235]]


In [42]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError
import traceback
import logging

# Run HTML features

In [32]:
## Idea: Combine with manual features from HTMLs

In [43]:
# HTML offline
import requests
from bs4 import BeautifulSoup
import re
# import whois
from urllib.parse import urlparse
from collections import Counter

# Function to parse HTML content from a URL
def get_html_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise error if the response is bad
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# Function to extract HTML structure features
def extract_structure_features(soup):
    features = {}
    
    features['num_divs'] = len(soup.find_all('div'))
    features['num_scripts'] = len(soup.find_all('script'))
    features['num_forms'] = len(soup.find_all('form'))
    features['num_links'] = len(soup.find_all('a'))
    features['num_iframes'] = len(soup.find_all('iframe'))
    
    # Text to HTML ratio
    text_length = len(soup.get_text())
    html_length = len(str(soup))
    features['text_to_html_ratio'] = text_length / html_length if html_length > 0 else 0
    
    return features

# Function to extract text-based features
def extract_text_features(soup):
    
    features = {}

    # Title tag content
    title = soup.title.string if soup.title and soup.title.string else ''
    features['title_length'] = len(title)

    # Visible text length
    visible_text = soup.get_text() if soup else ''
    features['visible_text_length'] = len(visible_text)

    return features

# Function to extract link-based features
# def extract_link_features(soup, base_url):
#     features = {}
    
#     all_links = soup.find_all('a', href=True)
#     internal_links = [link for link in all_links if urlparse(link['href']).netloc == urlparse(base_url).netloc]
#     external_links = [link for link in all_links if urlparse(link['href']).netloc != urlparse(base_url).netloc]
    
#     features['num_internal_links'] = len(internal_links)
#     features['num_external_links'] = len(external_links)
    
#     return features

from urllib.parse import urlparse

def extract_link_features(soup, base_url):
    features = {}
    
    try:
        all_links = soup.find_all('a', href=True)
        internal_links = []
        external_links = []
        
        for link in all_links:
            href = link['href']
            try:
                parsed_href = urlparse(href)
                parsed_base = urlparse(base_url)
                
                if parsed_href.netloc == parsed_base.netloc:
                    internal_links.append(href)
                else:
                    external_links.append(href)
            except ValueError:
                continue  # Skip invalid links
        
        features['num_internal_links'] = len(internal_links)
        features['num_external_links'] = len(external_links)
    except Exception as e:
        print(f"Error extracting link features: {e}")
        features['num_internal_links'] = 0
        features['num_external_links'] = 0
    
    return features


# Function to extract JavaScript-based features
def extract_js_features(soup):
    features = {}
    
    script_tags = soup.find_all('script')
    features['num_js_files'] = len(script_tags)
    
    # Suspicious JavaScript patterns
    suspicious_js_patterns = ['eval', 'document.write', 'window.location']
    features['suspicious_js_count'] = sum(
        any(p in script.get_text() for p in suspicious_js_patterns) for script in script_tags)
    
    return features

# Function to extract all features from a URL
def extract_features(url, rec_id):
#     html_content = get_html_content(url)
#     if not html_content:
#         return None
    
#     soup = BeautifulSoup(html_content, 'html.parser')
    
    f = codecs.open(Path(data_dir + '/' + generate_filename(rec_id)), 'r', encoding='utf-8', errors='ignore')
    soup = BeautifulSoup(f)
    
    # Extract all feature sets
    structure_features = extract_structure_features(soup)
    text_features = extract_text_features(soup)
    link_features = extract_link_features(soup, url)
    js_features = extract_js_features(soup)
    # domain_features = extract_domain_features(url)
    
    # Combine all features into a single dictionary
    features = {**structure_features, **text_features, **link_features, **js_features}
    
    return features

In [44]:
extract_features("http://vamoaestudiarmedicina.blogspot.com/", 1)

{'num_divs': 426,
 'num_scripts': 7,
 'num_forms': 0,
 'num_links': 121,
 'num_iframes': 0,
 'text_to_html_ratio': 0.23880716044000447,
 'title_length': 24,
 'visible_text_length': 44917,
 'num_internal_links': 39,
 'num_external_links': 79,
 'num_js_files': 7,
 'suspicious_js_count': 0}

In [45]:
smalldata.iloc[159,0]

7605

In [46]:
smalldata.iloc[159,1]

'https://zpr.io/FxiKGNiWpd3T'

In [ ]:
sma

In [47]:
manual_html_features = []
for idx in range(len(smalldata.index)):
    url = smalldata.iloc[idx, 1]
    print(idx,end=',')
    rec_id = smalldata.iloc[idx, 0]
    features = extract_features(url, rec_id)
    manual_html_features.append(list(features.values()))

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [48]:
import pickle

# Lưu vào file pickle
with open("manual_html_features_URLdataset13.2.pkl", "wb") as f:
    pickle.dump(manual_html_features, f)

# Mở file pickle và nạp dữ liệu vào biến manual_html_features
# with open("manual_html_features_URLdataset13.2.pkl", "rb") as f:
#     manual_html_features_13.2 = pickle.load(f)

In [49]:
len(manual_html_features)

2231

In [50]:
run_ML(np.array(manual_html_features), labels, "URLdataset13.2")

Run: 0, fold: 0
LightGBM, Fold 0 - F1 Score: 0.8258388, Precision: 0.8235562, Recall: 0.8283076
Run: 0, fold: 1
LightGBM, Fold 1 - F1 Score: 0.8079793, Precision: 0.8275653, Recall: 0.7956164
Run: 0, fold: 2
LightGBM, Fold 2 - F1 Score: 0.8579392, Precision: 0.8647378, Recall: 0.8523317
Run: 0, fold: 3
LightGBM, Fold 3 - F1 Score: 0.8524953, Precision: 0.8448440, Recall: 0.8622484
Run: 0, fold: 4
LightGBM, Fold 4 - F1 Score: 0.8538467, Precision: 0.8491075, Recall: 0.8603147
['LightGBM']
F1 Score, Precision, Recall: [[0.8396199 0.8419622 0.8397638]]


In [55]:
urlhtml_manual_features = np.concatenate([phish_url, manual_html_features], axis=1)

In [56]:
ngramUrl_manualHtml_features = np.concatenate([phish_url_countvectorizer, manual_html_features], axis=1)

In [57]:
run_ML(np.array(urlhtml_manual_features), labels, "URLdataset132", "urlhtml")

Run: 0, fold: 0
LightGBM, Fold 0 - F1 Score: 0.8845352, Precision: 0.8824732, Recall: 0.8867121
Run: 0, fold: 1
LightGBM, Fold 1 - F1 Score: 0.8962911, Precision: 0.9045304, Recall: 0.8893836
Run: 0, fold: 2
LightGBM, Fold 2 - F1 Score: 0.8998243, Precision: 0.9142432, Recall: 0.8891873
Run: 0, fold: 3
LightGBM, Fold 3 - F1 Score: 0.9035123, Precision: 0.8969601, Recall: 0.9111707
Run: 0, fold: 4
LightGBM, Fold 4 - F1 Score: 0.9071505, Precision: 0.9083158, Recall: 0.9060315
['LightGBM']
F1 Score, Precision, Recall: [[0.8982627 0.9013045 0.896497 ]]


### Create dataset class for PyG

In [58]:
import torch
from torch_geometric.data import Data

In [72]:
if True: #don't delete this
    graphs = []
    labels_list = []
    for i in range(0,len(smalldata.index)):
    # for i in range(2):
        nodes_list = []
        attr_list = []
        print(i, end = ',')
        X, A, y, X_A, _ = get_udst_inputs(smalldata.iloc[i, 1], smalldata.iloc[i, 0], smalldata.iloc[i, 2])

        child_id, source_id = A.nonzero()
        edge_index = torch.tensor([source_id,
                                   child_id], dtype=torch.long)
        x = torch.tensor(X, dtype=torch.float)
        y = torch.tensor([labels[i]], dtype=torch.int64)
        data = Data(x=x, edge_index=edge_index, y = y)
        graphs.append(data)
        labels_list.append(labels[i])

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,27

In [73]:
len(graphs)

2231

In [74]:
import pickle
with open("graph_html_13.2_final_suf", "wb") as fp:   #Pickling
    pickle.dump(graphs, fp)
with open("graph_html_labels_13.2_final_suf", "wb") as fp:   #Pickling
    pickle.dump(labels_list, fp)
    
# with open("graph_html", "rb") as fp:   # Unpickling
#     graphs = pickle.load(fp)
# with open("graph_html_labels", "rb") as fp:   # Unpickling
#     labels_list = pickle.load(fp)

In [75]:
print("Train freq: ", [len(list(group)) for key, group in groupby(sorted(labels_list))])

Train freq:  [1496, 735]


In [76]:
import torch
from torch_geometric.data import Data, Dataset

class GraphClassificationDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs
        # self.labels = labels

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        # label = self.labels[idx]
        return graph
    
    def get(): pass

    def len(): pass

In [77]:
dataset = GraphClassificationDataset(graphs)

In [78]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.


Dataset: GraphClassificationDataset(2231):
Number of graphs: 2231
Number of features: 2
Number of classes: 2


In [81]:
# train_dataset = dataset[:int(0.8*n_samples)]
# test_dataset = dataset[int(0.8*n_samples):]
train_dataset = [dataset[idx] for idx in train_idx]
test_dataset = [dataset[idx] for idx in test_idx]

In [82]:
len(train_dataset), len(test_dataset)

(1784, 447)

In [83]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### Build and train PyG

In [84]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)
        self.linconcat = Linear(2*hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

In [169]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    # for data in loader:  # Iterate in batches over the training/test dataset.
    #     out = model(data.x, data.edge_index, data.batch)  
    #     pred = out.argmax(dim=1)  # Use the class with highest probability.
    #     correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    # return correct / len(loader.dataset)  # Derive ratio of correct predictions.
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
        # print(pred_labels)
    return f1_score(true_labels, pred_labels, average='macro')

for epoch in range(0, 20):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

Epoch: 000, Train F1: 0.4124, Test F1: 0.4148
Epoch: 010, Train F1: 0.4227, Test F1: 0.4402


### GNN for dim. reduction

#### Net 1: 0.87, 0.88

In [85]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class GCNdimReduce(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNdimReduce, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin1(x)
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        # x = x.relu()
        # x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # # 3. Apply a final classifier
        x = self.lin1(x)
        return x

#### Net 2

In [41]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool, global_max_pool

class GCNdimReduceV2(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNdimReduceV2, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
       
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        # 3. Apply a final classifier
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        
        return x
    
    def dimReduce(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin1(x)
        return x

#### Test model

In [86]:
n_hidden_channels = 16
model = GCNdimReduce(hidden_channels=n_hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        # print(data.x.shape)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()
    correct = 0
    true_labels = []
    pred_labels = []
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        true_labels += data.y.tolist()
        pred_labels += pred.tolist()
    return f1_score(true_labels, pred_labels, average='macro')

for epoch in range(0, 10):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')

Epoch: 000, Train F1: 0.4041, Test F1: 0.3902
Epoch: 001, Train F1: 0.4041, Test F1: 0.3902
Epoch: 002, Train F1: 0.4041, Test F1: 0.3902
Epoch: 003, Train F1: 0.4041, Test F1: 0.3902
Epoch: 004, Train F1: 0.4041, Test F1: 0.3902
Epoch: 005, Train F1: 0.4041, Test F1: 0.3902
Epoch: 006, Train F1: 0.4041, Test F1: 0.3902
Epoch: 007, Train F1: 0.4041, Test F1: 0.3902
Epoch: 008, Train F1: 0.4079, Test F1: 0.3969
Epoch: 009, Train F1: 0.4522, Test F1: 0.4418


#### Def

In [87]:
def train_model(all_data_loader, train_loader, test_loader, n_hidden_channels = 16, n_epoch=1, lr=0.001):
    # n_hidden_channels = 16
    model = GCNdimReduce(hidden_channels=n_hidden_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    def train():
        model.train()

        for data in train_loader:  # Iterate in batches over the training dataset.
            # print(data.x.shape)
            out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
            loss = criterion(out, data.y)  # Compute the loss.
            loss.backward()  # Derive gradients.
            optimizer.step()  # Update parameters based on gradients.
            optimizer.zero_grad()  # Clear gradients.

    def test(loader):
        model.eval()
        correct = 0
        true_labels = []
        pred_labels = []
        for data in loader:  # Iterate in batches over the training/test dataset.
            out = model(data.x, data.edge_index, data.batch)  
            pred = out.argmax(dim=1)  # Use the class with highest probability.
            correct += int((pred == data.y).sum())  # Check against ground-truth labels.
            true_labels += data.y.tolist()
            pred_labels += pred.tolist()
        return f1_score(true_labels, pred_labels, average='macro')

    for epoch in range(0, n_epoch):
        train()
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        if epoch % 1 == 0:
            print(f'Epoch: {epoch:03d}, Train F1: {train_acc:.4f}, Test F1: {test_acc:.4f}')
    
    model.eval()
    dim_vec = torch.empty((0, n_hidden_channels), dtype=torch.float32)
    for data in all_data_loader:
        dim_x = model.dimReduce(data.x, data.edge_index, data.batch)
        dim_vec = torch.cat((dim_vec, dim_x), 0)
    return (dim_vec)

In [88]:
all_data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
dim_vec = train_model(all_data_loader, train_loader, test_loader)

Epoch: 000, Train F1: 0.4041, Test F1: 0.3902


In [89]:
len(dataset)

2231

In [90]:
path_dir="results"

In [ ]:
def runHUGPHISH(train_idx, test_idx):
    # stack_GNNs_graph = np.concatenate((np.array(phish_url_vectorizer), hyperlink_features),axis=1)
    stack_GNNs_graph = np.array(ngramUrl_manualHtml_features)
    for i in range(10):
        np.random.seed(i) 
        # train_idx_new = list(np.random.choice(list(range(n_samples)), int(0.8*n_samples), replace=False))
        # train_idx_new = list(set(train_idx_new).difference(test_idx))
        train_idx_new = list(np.random.choice(train_idx, int(0.8*len(train_idx)), replace=False))
        print(train_idx_new[:5])
        train_dataset_new = [dataset[idx] for idx in train_idx_new]
        train_loader_new = DataLoader(train_dataset_new, batch_size=8, shuffle=True)
        # test_dataset = [dataset[idx] for idx in test_idx]
        # test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
        n_hidden_channels = 6
        n_epoch = 1
        dim_vec_new = train_model(all_data_loader, train_loader_new, test_loader, n_hidden_channels, n_epoch)
        stack_GNNs_graph = np.concatenate((stack_GNNs_graph, dim_vec_new.detach().numpy()),axis=1)
    
    model_lgb2 = lgb.LGBMClassifier(verbose=-1)
    model_lgb2.fit(stack_GNNs_graph[train_idx], labels[train_idx])
    y_predict=model_lgb2.predict(stack_GNNs_graph[test_idx]) 
    

In [94]:
runHUGPHISH(train_idx, test_idx)

[2027, 71, 1034, 862, 1603]
Epoch: 000, Train F1: 0.4012, Test F1: 0.3902
[772, 829, 161, 478, 1425]
Epoch: 000, Train F1: 0.4081, Test F1: 0.3902
[217, 1010, 1226, 2032, 1511]
Epoch: 000, Train F1: 0.4019, Test F1: 0.3902
[1050, 539, 246, 519, 1837]
Epoch: 000, Train F1: 0.4052, Test F1: 0.3902
[743, 2197, 2199, 1132, 241]
Epoch: 000, Train F1: 0.4044, Test F1: 0.3902
[440, 1030, 1419, 583, 143]
Epoch: 000, Train F1: 0.4032, Test F1: 0.3902
[1503, 2042, 1942, 1276, 838]
Epoch: 000, Train F1: 0.3999, Test F1: 0.3902
[825, 762, 1851, 1108, 1515]
Epoch: 000, Train F1: 0.4037, Test F1: 0.3902
[747, 1993, 561, 1282, 169]
Epoch: 000, Train F1: 0.4044, Test F1: 0.3902
[2119, 1293, 1529, 309, 806]
Epoch: 000, Train F1: 0.4049, Test F1: 0.3902
0.9100813883316208


,true_label,pred_label,predict_proba
3,0,0,0.000390
7,0,0,0.000081
2059,0,0,0.032668
2065,1,1,0.998089
2067,1,1,0.998608
...,...,...,...
2036,1,1,0.999522
2037,1,1,0.975015
2040,1,1,0.919113
2043,0,0,0.057086


In [109]:
n_loops = 1; n_folds = 5;
base_dir = 'results'
approach = 'HUGPhish'
dataset1 = 'URLdataset13.2'
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
for i in range(n_loops):
    cv = KFold(n_splits=n_folds, shuffle=True, random_state = i)
    # print("Use stratifired fold")
    # cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state = i)
    for fold, (train_idx, test_idx) in enumerate(cv.split(smalldata)):
        path_dir = base_dir +'/' + dataset1 + '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_'+approach
        print('Run: ', i, ', fold: ', fold)
        # X_train = X[train_idx]
        # X_test = X[test_idx]
        # y_train = y[train_idx]
        # y_test = y[test_idx]
       
        df = runHUGPHISH(train_idx, test_idx)
        path_dir = base_dir +'/' + dataset1 + '_run_'+str(i)+'_'+ 'fold_'+str(fold)+'_'+approach
        df.to_csv(path_dir + '_labels.csv', index=False)

Run:  0 , fold:  0
[1271, 1188, 1498, 1214, 1794]
Epoch: 000, Train F1: 0.4024, Test F1: 0.3902
[1941, 1081, 380, 136, 2146]
Epoch: 000, Train F1: 0.3948, Test F1: 0.3902
[784, 649, 644, 1012, 1370]
Epoch: 000, Train F1: 0.3987, Test F1: 0.3902
[782, 1123, 2206, 978, 456]
Epoch: 000, Train F1: 0.3994, Test F1: 0.3902
[859, 2187, 327, 1206, 1331]
Epoch: 000, Train F1: 0.4004, Test F1: 0.3902
[152, 1848, 1597, 804, 1805]
Epoch: 000, Train F1: 0.4014, Test F1: 0.3902
[1808, 830, 113, 177, 888]
Epoch: 000, Train F1: 0.4019, Test F1: 0.3902
[81, 356, 1392, 583, 999]
Epoch: 000, Train F1: 0.3992, Test F1: 0.3902
[1963, 244, 1932, 1912, 2100]
Epoch: 000, Train F1: 0.4012, Test F1: 0.3902
[1917, 378, 526, 516, 779]
Epoch: 000, Train F1: 0.3974, Test F1: 0.3902
0.9000117730162468
Run:  0 , fold:  1
[1253, 1174, 1472, 1200, 1783]
Epoch: 000, Train F1: 0.4057, Test F1: 0.3902
[1937, 1059, 382, 137, 1705]
Epoch: 000, Train F1: 0.4023, Test F1: 0.3902
[754, 624, 621, 994, 1354]
Epoch: 000, Train F1